In [1]:
%run import_libs.py

### get data

In [2]:
short_list = [
    'P_2_last', 'B_1_last', 'B_3_last', 'D_39_last', 'D_41_last', 'D_45_max', 'B_28_min', 'D_42_avg', 'B_14_last',
    'D_50_last','B_5_last','B_4_last','D_44_last','P_2_min','D_46_last','D_51_avg','B_2_avg','P_2_avg','D_49_last',
    'S_3_last','S_3_avg','D_43_avg','B_4_max', 'D_39_max', 'D_43_last', 'S_5_last', 'B_7_last', 'B_18_last', 'P_3_avg', 
    'B_9_last', 'R_4_last', 'R_2_last', 'D_121_min', 'S_26_max'
]

In [4]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')
num_features = pd.read_csv("num_feats_after_filtering.csv")["0"].to_list()
# num_features = short_list

df_train_agg = get_df_w_aggrs(df=df_train, feats=num_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

(458913, 151)
(458913, 151)
(458913, 151)
(458913, 162)
(458913, 616)


In [9]:
df_test = get_test_data(TEST_PATH='./data/test.parquet')
df_test = get_df_w_aggrs(df=df_test, feats=num_features)

(924621, 151)
(924621, 151)
(924621, 151)
(924621, 162)
(924621, 616)


In [ ]:
# оставляем только short_list
df_train = df_train[['customer_ID'] + short_list + ['target']]
df_test = df_test[['customer_ID'] + short_list]

### fit model

In [17]:
len(short_list)

34

In [16]:
# fit predict engeneering
random_state = 42
group_col = 'customer_ID'
target_col = 'target'
y, groups = df_train[target_col].values, df_train[group_col].values
fold_scores, fold_test_raw = [], []

sgkf = StratifiedGroupKFold(5, shuffle=True, random_state=random_state)

params = dict(
    learning_rate=.05, subsample=.9, subsample_freq=1,
    colsample_bytree=.8, objective='binary',
    random_state=random_state,
    max_depth=7, n_estimators=484,
    n_jobs=-1, verbosity=-1
)

for fold, (tr_idx, va_idx) in enumerate(
        sgkf.split(df_train[[group_col, target_col]], y, groups)):
    print(f"Fold {fold}")
    X_tr, X_va = df_train.iloc[tr_idx][short_list], df_train.iloc[va_idx][short_list]
    y_tr, y_va = y[tr_idx], y[va_idx]
    model = LGBMClassifier(**params)
    model.fit(X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                eval_metric=lgb_amex_metric)
    pred_va = model.predict_proba(X_va)[:,1]
    sc = get_amex_metric_calculated(y_va, pred_va)
    print(f" Amex = {sc:.5f}")
    fold_scores.append(sc)
    fold_test_raw.append(model.predict_proba(df_test[short_list], raw_score=True))

Fold 0
 Amex = 0.77913
Fold 1
 Amex = 0.78145
Fold 2
 Amex = 0.78321
Fold 3
 Amex = 0.77928
Fold 4
 Amex = 0.78402


In [18]:
mean_cv = float(np.mean(fold_scores))
print(f"Mean CV Amex = {mean_cv:.5f}")

Mean CV Amex = 0.78142


In [ ]:
# fit on all train
# shap

### hyperparameter search (optuna)

In [ ]:
# optuna

### interpreting results